In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [3]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]
#dodac exponential

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.0957741  0.0378656    0.202848    …  0.109026     0.092351  -0.051267
 -0.0266204  0.0049847   -0.0150684      0.0442945   -0.027667  -0.0243223
 -0.003398   0.00583799  -0.00317517     0.00176932   0.003932   0.00661877

[:, :, 3] =
 -0.0141912    0.189499     0.111722    …  0.0337892   0.00856265  0.136876
  0.00620094  -0.0277979   -0.0590367      0.0273218  -0.0289399   0.024985
 -0.0107861    0.00492943   0.00858379     0.0113027  -0.00063482  0.015204

;;; … 

[:, :, 59] =
 -0.050804    -0.151979   -0.0180691  …   0.10875     0.115645    0.066843
  0.114153     0.0107052   0.102025      -0.022995    0.00197505  0.0585414
  0.00603242   0.0181106   0.0146704      0.00440294  0.00784149  0.00687073

[:, :, 60] =
 -0.0189

In [4]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [5]:
returns_scenarios

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.0957741  0.0378656    0.202848    …  0.109026     0.092351  -0.051267
 -0.0266204  0.0049847   -0.0150684      0.0442945   -0.027667  -0.0243223
 -0.003398   0.00583799  -0.00317517     0.00176932   0.003932   0.00661877

[:, :, 3] =
 -0.0141912    0.189499     0.111722    …  0.0337892   0.00856265  0.136876
  0.00620094  -0.0277979   -0.0590367      0.0273218  -0.0289399   0.024985
 -0.0107861    0.00492943   0.00858379     0.0113027  -0.00063482  0.015204

;;; … 

[:, :, 59] =
 -0.050804    -0.151979   -0.0180691  …   0.10875     0.115645    0.066843
  0.114153     0.0107052   0.102025      -0.022995    0.00197505  0.0585414
  0.00603242   0.0181106   0.0146704      0.00440294  0.00784149  0.00687073

[:, :, 60] =
 -0.0189

In [6]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)

lattice = cluster(s,9)

lattice[1]

9×3 transpose(::Matrix{Float64}) with eltype Float64:
  0.293238   -0.025197    0.0118398
  0.199479   -0.133125   -0.0199864
  0.104999    0.0153817  -0.017483
 -0.0850867   0.148748    0.0368125
 -0.0767366  -0.0773371  -0.0254048
  0.181435    0.0821686   0.0434085
 -0.163336    0.0640253  -7.26403e-5
  0.0906274   0.107159    0.00942499
  0.0235733  -0.0359247   0.0168708

In [7]:
s

3×100000 Matrix{Float64}:
  0.10528     -0.0338666  -0.0419397  …   0.389978    -0.171615   0.240585
  0.0625979    0.126367    0.0909913     -0.0857693   -0.077198   0.219658
 -0.00193111   0.0486711   0.0450692      0.00927519   0.0101065  0.048267

In [8]:
clusters_probs(lattice[2])

9-element Vector{Float64}:
 0.116
 0.09516
 0.12167
 0.0941
 0.09343
 0.09178
 0.11544
 0.12098
 0.15144

In [21]:
res = generate_lattice(returns_scenarios, 5,5)

res[1].states[1,:,:]

5×3 Matrix{Float64}:
  0.170277    0.0598518   0.0284973
  0.0418266  -0.0961852  -0.0269954
 -0.0636481   0.0253821  -0.00190151
 -0.0743621   0.141428    0.0302314
  0.234949   -0.0542054  -3.58034e-5

In [22]:
res[2]

5-element Vector{Any}:
 0.21594
 0.15243
 0.27314
 0.15206
 0.20643